In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import seaborn as sn
import teotil2 as teo

sn.set_context("notebook")

# Estimating loads in unmonitored regions - 2021

This notebook uses the [TEOTIL2 model](https://nivanorge.github.io/teotil2/) to estimate loads in unmonitored areas. We know the regine ID for each of the 155 stations where water chemistry is measured, and we also know which OSPAR region each monitoring site drains to. We want to use observed data to estimate loads upstream of each monitoring point, and modelled data elsewhere.

In [2]:
# Connect to db
engine = nivapy.da.connect()

Username:  ········
Password:  ········


Connection successful.


## 1. Generate model input file

In [3]:
# Year of interest
year = 2021

# Parameters of interest
par_list = ["Tot-N", "Tot-P"]

# Path to TEOTIL2 "core" input data
teo_fold = r"../../../../teotil2/data/core_input_data"

# Ouput path for model file
ann_input_csv = f"../../../../teotil2/data/norway_annual_input_data/input_data_{year}.csv"

In [4]:
# Make input file
df = teo.io.make_input_file(
    year, engine, teo_fold, ann_input_csv, mode="nutrients", par_list=par_list
)

/opt/conda/lib/python3.10/site-packages/teotil2/io.py:731: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(value=0, inplace=True)


## 2. Run model

In [5]:
%%time

# Run model
g = teo.model.run_model(ann_input_csv)

/opt/conda/lib/python3.10/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()


CPU times: user 4.51 s, sys: 44.8 ms, total: 4.55 s
Wall time: 4.59 s


## 3. Save results

In [6]:
# Save results as csv
res_csv = f"../../../../teotil2/data/norway_annual_output_data/teotil2_results_{year}.csv"
df = teo.model.model_to_dataframe(g, out_path=res_csv)

df.head()

,regine,regine_ned,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,...,local_ren_tot-n_tonnes,local_ren_tot-p_tonnes,local_runoff_mm/yr,local_spr_tot-n_tonnes,local_spr_tot-p_tonnes,local_trans_tot-n,local_trans_tot-p,local_urban_tot-n_tonnes,local_urban_tot-p_tonnes,local_vol_lake_m3
0,001.1A2B,001.1A2A,2.950585,0.199769,0.033277,0.002690,0.978313,0.013494,13.657143,0.525534,...,0.7092,0.00038,262.607398,0.235836,0.010424,1.00,1.00,0.02205,0.00315,1.788754e+08
1,001.1A4D,001.1A4C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.235919,0.008714,...,0.0000,0.00000,306.375298,0.000000,0.000000,0.81,0.26,0.00000,0.00000,8.019497e+06
2,001.1M,001.1L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.090878,0.032044,...,0.0000,0.00000,306.375298,0.000000,0.000000,0.86,0.31,0.00000,0.00000,3.866908e+07
3,001.221Z,001.2210,0.119042,0.008060,0.001343,0.000109,0.010857,0.000529,0.367738,0.011287,...,0.0000,0.00000,262.607398,0.009515,0.000421,1.00,1.00,0.00000,0.00000,0.000000e+00
4,001.222Z,001.2220,0.980964,0.066416,0.011063,0.000894,0.089471,0.004360,2.027571,0.082984,...,0.0000,0.00000,262.607398,0.078407,0.003466,1.00,1.00,0.00000,0.00000,0.000000e+00


In [7]:
# Save version with main catchments only
main_list = ["%03d." % i for i in range(1, 316)]
df2 = df.query("regine in @main_list")
df2.sort_values("regine", inplace=True)

# Save
main_csv = f"../../../../Results/Unmon_loads/teotil2_results_{year}_main_catchs.csv"
df2.to_csv(main_csv, index=False, encoding="utf-8")

/tmp/ipykernel_2709/1277450711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.sort_values("regine", inplace=True)


## 4. Explore results

### 4.1. Total N and P

####  4.1.1. Identify areas with monitoring data

Where observations are available, we want to use them in preference to the model output. This means identifying all the catchments with observed data and substracting the model results for these locations. This is more complicated than it appears, because a small number of observed catchments are upstream of others, so subtracting all the loads for the 155 monitored catchments involves "double accounting", which we want to avoid. The first step is therefore to identify the downstream-most nodes for the monitored areas i.e. for the cases where one catchment is upstream of another, we just want the downstream node.

In [8]:
# Read station data
in_xlsx = r"../../../../Data/RID_Sites_List_2017-2020.xlsx"
stn_df = pd.read_excel(in_xlsx, sheet_name="RID_All")

# Get just cols of interest and drop duplicates
# (some sites are in the same regine)
stn_df = stn_df[["ospar_region", "nve_vassdrag_nr"]].drop_duplicates()

# Get catch IDs with obs data
obs_nds = set(stn_df["nve_vassdrag_nr"].values)

# Build network from input file
g, nd_list = teo.calib.build_calib_network(ann_input_csv, obs_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df["nve_vassdrag_nr"].isin(ds_nds)]

#### 4.1.2. Sum model results for monitored locations

In [9]:
# Read model output
teo_df = pd.read_csv(res_csv)

# Join accumulated outputs to stns of interest
mon_df = pd.merge(
    stn_df, teo_df, how="left", left_on="nve_vassdrag_nr", right_on="regine"
)

# Groupby OSPAR region
mon_df = mon_df.groupby("ospar_region").sum()

# Get just accum cols
cols = [i for i in mon_df.columns if i.split("_")[0] == "accum"]
mon_df = mon_df[cols]

mon_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,148.332680,4.979030,2.550150,0.219317,84.799209,4.375319,3559.169271,50.938802,148.332680,4.979030,...,3326.037382,41.584453,758.132171,33.895765,0.889819,48.353294,3.266183,63712.78,0.000000,0.000000
NORTH SEA,2907.533597,64.687665,34.862152,2.176470,383.398455,34.670842,10680.060828,160.433912,2913.506167,65.231141,...,7383.156206,60.531929,991.033928,175.697323,18.358870,133.542671,8.102523,23353.19,5.972570,0.543476
NORWEGIAN SEA2,2953.759656,80.839590,40.763819,3.061545,611.520213,52.618116,11217.184832,241.763176,2970.226634,82.968741,...,7635.437985,106.176319,1406.381002,308.486023,18.733922,196.468190,19.123709,45896.63,16.466978,2.129151
SKAGERAK,11451.659078,266.027650,102.079627,5.546824,3822.641631,98.114091,28863.598352,504.407758,11613.594317,283.505413,...,13427.362404,122.788254,1686.803348,2751.736182,21.026259,795.225408,40.195555,93945.27,161.935239,17.477763


This table gives the **modelled** inputs to each OSPAR region from catchments for which we have observed data. We want to subtract these values from the overall modelled inputs to each region and substitute the observed data instead.

The trickiest part of this is that the OSPAR regions in the TEOTIL catchment network (files named `regine_{year}.csv`) don't exactly match the relevant OSPAR definitions for this analysis. This is because the "OSPAR boundaries" in the model include catchments draining to Sweden (as part of TEOTIL2 Metals - see [here](https://nivanorge.github.io/teotil2/pages/07_1000_lakes.html)), so instead of using them directly we need to aggregate based on vassdragsnummers.

#### 4.1.3. Group model output according to "new" OSPAR regions

In [10]:
# Define "new" OSPAR regions (ranges are inclusive)
os_dict = {
    "SKAGERAK": (1, 23),
    "NORTH SEA": (24, 90),
    "NORWEGIAN SEA2": (91, 170),
    "LOFOTEN-BARENTS SEA": (171, 247),
}

# Container for results
df_list = []

# Loop over model output
for reg in os_dict.keys():
    min_id, max_id = os_dict[reg]

    regs = ["%03d." % i for i in range(min_id, max_id + 1)]

    # Get data for this region
    df2 = teo_df[teo_df["regine"].isin(regs)]

    # Get just accum cols
    cols = [i for i in df2.columns if i.split("_")[0] == "accum"]
    df2 = df2[cols]

    # Add region
    df2["ospar_region"] = reg

    # Add to output
    df_list.append(df2)

# Build df
os_df = pd.concat(df_list, axis=0)

# Aggregate
os_df = os_df.groupby("ospar_region").sum()

os_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,618.080639,25.476645,10.565114,0.885819,20608.026007,3440.248883,28980.886317,3585.502919,628.052949,26.880485,...,7744.807361,118.373551,1974.326290,1210.344048,134.226598,314.365113,34.266749,138090.89,9.972310,1.403841
NORTH SEA,6952.448325,177.329876,80.323926,5.829535,25189.349730,4181.386083,47100.548125,4501.838882,6992.265806,182.398546,...,14918.932589,138.054254,2068.874479,3074.240824,418.676018,678.740391,74.235457,59314.38,39.817482,5.068670
NORWEGIAN SEA2,8006.530178,260.884364,101.708615,8.082202,35330.714040,5884.984039,57642.808732,6360.244827,8038.688969,265.124807,...,14273.405723,210.135981,2802.512631,2633.093093,344.224601,675.837568,76.851612,113934.05,32.158791,4.240442
SKAGERAK,13174.740205,338.407599,112.985724,6.352341,10087.936491,229.697682,38047.528909,735.559212,13448.474673,371.139565,...,14511.117744,134.721965,1783.421413,7931.117678,93.608899,938.298190,53.478840,102574.69,273.734469,32.731966


We can now calculate the unmonitored component by simply subtracting the values modelled upstream of monitoring stations from the overall modelled inputs to each OSPAR region.

#### 4.1.4. Estimate loads in unmonitored areas

In [11]:
# Calc unmonitored loads
unmon_df = os_df - mon_df

# Write output
out_csv = f"../../../../Results/Unmon_loads/teotil2_raw_unmonitored_loads_{year}.csv"
unmon_df.to_csv(out_csv, encoding="utf-8", index_label="ospar_region")

unmon_df.round(0).astype(int).T

ospar_region,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
accum_agri_diff_tot-n_tonnes,470,4045,5053,1723
accum_agri_diff_tot-p_tonnes,20,113,180,72
accum_agri_pt_tot-n_tonnes,8,45,61,11
accum_agri_pt_tot-p_tonnes,1,4,5,1
accum_all_point_tot-n_tonnes,20523,24806,34719,6265
accum_all_point_tot-p_tonnes,3436,4147,5832,132
accum_all_sources_tot-n_tonnes,25422,36420,46426,9184
accum_all_sources_tot-p_tonnes,3535,4341,6118,231
accum_anth_diff_tot-n_tonnes,480,4079,5068,1835
accum_anth_diff_tot-p_tonnes,22,117,182,88


#### 4.1.5. Aggregate values to required quantities

In [12]:
# Aggregate to match report
unmon_df["flow"] = unmon_df["accum_q_m3/s"] * 60 * 60 * 24 / 1000.0  # 1000s m3/day

unmon_df["sew_n"] = (
    unmon_df["accum_ren_tot-n_tonnes"] + unmon_df["accum_spr_tot-n_tonnes"]
)
unmon_df["sew_p"] = (
    unmon_df["accum_ren_tot-p_tonnes"] + unmon_df["accum_spr_tot-p_tonnes"]
)

unmon_df["ind_n"] = unmon_df["accum_ind_tot-n_tonnes"]
unmon_df["ind_p"] = unmon_df["accum_ind_tot-p_tonnes"]

unmon_df["fish_n"] = unmon_df["accum_aqu_tot-n_tonnes"]
unmon_df["fish_p"] = unmon_df["accum_aqu_tot-p_tonnes"]

unmon_df["diff_n"] = (
    unmon_df["accum_anth_diff_tot-n_tonnes"] + unmon_df["accum_nat_diff_tot-n_tonnes"]
)
unmon_df["diff_p"] = (
    unmon_df["accum_anth_diff_tot-p_tonnes"] + unmon_df["accum_nat_diff_tot-p_tonnes"]
)

new_df = unmon_df[
    ["flow", "sew_n", "sew_p", "ind_n", "ind_p", "fish_n", "fish_p", "diff_n", "diff_p"]
]

# Total for Norway
new_df.loc["NORWAY"] = new_df.sum(axis=0)

# Reorder rows
new_df = new_df.reindex(
    ["NORWAY", "LOFOTEN-BARENTS SEA", "NORTH SEA", "NORWEGIAN SEA2", "SKAGERAK"]
)

new_df.round().astype(int)

/tmp/ipykernel_2709/2545542187.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc["NORWAY"] = new_df.sum(axis=0)


,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p
ospar_region,,,,,,,,,
NORWAY,327178,13013,1100,2360,227,70816,12210,31138,679
LOFOTEN-BARENTS SEA,105079,1442,164,125,11,18948,3260,4898,99
NORTH SEA,93125,3444,466,441,83,20875,3594,11615,195
NORWEGIAN SEA2,120626,2804,383,899,95,30955,5349,11706,286
SKAGERAK,8348,5322,86,895,38,37,6,2919,100


## 5. Other N and P species

Tore's procedure `RESA2.FIXTEOTILPN` defines simple correction factors for estimating PO4, NO3 and NH4 from total P and N. The table below lists the factors used.

|   Source    | Phosphate | Nitrate | Ammonium |
|:-----------:|:---------:|:-------:|:--------:|
|    Sewage   |     0.600 |   0.050 |    0.750 |
|   Industry  |     0.600 |   0.050 |    0.750 |
| Aquaculture |     0.690 |   0.110 |    0.800 |
|   Diffuse   |     0.246 |   0.625 |    0.055 |


In [13]:
# Dict of conversion factors
con_dict = {
    ("sew", "po4"): ("p", 0.6),
    ("ind", "po4"): ("p", 0.6),
    ("fish", "po4"): ("p", 0.69),
    ("diff", "po4"): ("p", 0.246),
    ("sew", "no3"): ("n", 0.05),
    ("ind", "no3"): ("n", 0.05),
    ("fish", "no3"): ("n", 0.11),
    ("diff", "no3"): ("n", 0.625),
    ("sew", "nh4"): ("n", 0.75),
    ("ind", "nh4"): ("n", 0.75),
    ("fish", "nh4"): ("n", 0.8),
    ("diff", "nh4"): ("n", 0.055),
}

# Apply factors
for src in ["sew", "ind", "fish", "diff"]:
    for spc in ["po4", "no3", "nh4"]:
        el, fac = con_dict[(src, spc)]
        new_df[src + "_" + spc] = fac * new_df[src + "_" + el]

new_df.round().astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,327178,105079,93125,120626,8348
sew_n,13013,1442,3444,2804,5322
sew_p,1100,164,466,383,86
ind_n,2360,125,441,899,895
ind_p,227,11,83,95,38
fish_n,70816,18948,20875,30955,37
fish_p,12210,3260,3594,5349,6
diff_n,31138,4898,11615,11706,2919
diff_p,679,99,195,286,100
sew_po4,660,99,280,230,52


## 6. Other quantities

The model currently only considers N and P, but the project focuses on a wider range of parameters. For now, we simply assume that all measured inputs (`renseanlegg`, `industri` and `akvakultur`) for regines outside of catchments with measured data make it to the sea.

We only want data for catchments that are not monitored i.e. for regine IDs **not** in the graph created above.

In [14]:
# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = (
    "SELECT SUBSTR(a.regine, 1, 3) AS vassdrag, "
    "  a.type, "
    "  b.name, "
    "  b.unit, "
    "  SUM(c.value * d.factor) as value "
    "FROM RESA2.RID_PUNKTKILDER a, "
    "RESA2.RID_PUNKTKILDER_OUTPAR_DEF b, "
    "RESA2.RID_PUNKTKILDER_INPAR_VALUES c, "
    "RESA2.RID_PUNKTKILDER_INP_OUTP d "
    "WHERE a.anlegg_nr = c.anlegg_nr "
    "AND (1, a.regine) NOT IN %s "
    "AND d.in_pid = c.inp_par_id "
    "AND d.out_pid = b.out_pid "
    "AND c.year = %s "
    "GROUP BY SUBSTR(a.regine, 1, 3), a.type, b.name, b.unit "
    "ORDER BY SUBSTR(a.regine, 1, 3), a.type" % (tuple(nd_list_hack), year)
)

df = pd.read_sql(sql, engine)

# Tidy
df["par"] = df["type"] + "_" + df["name"] + "_" + df["unit"]
del df["name"], df["unit"], df["type"]

# Pivot
df = df.pivot(index="vassdrag", columns="par", values="value")
df.reset_index(inplace=True)

In [15]:
def f(x):
    try:
        a = int(x)
        return a
    except:
        return -999


# Convert vassdrag to numbers
df["vass"] = df["vassdrag"].apply(f)

# Get just the main catchments
df = df.query("vass != -999")

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PCB_tonn,...,RENSEANLEGG_Hg_tonn,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass
0,001,0.019290,1.362000e-02,0.007670,0.085870,0.000000,NaN,NaN,0.043210,NaN,...,0.000011,0.012184,0.000001,0.000000,0.000801,77.72544,125.01180,1.22541,0.057156,1
1,002,0.011934,6.225700e-03,0.899602,2.712862,0.003595,NaN,NaN,0.654341,NaN,...,0.000062,0.196834,0.000246,0.000000,0.007702,383.96749,729.93910,5.62835,0.737233,2
2,003,0.000012,6.000000e-07,0.000054,0.000076,0.000000,0.0,0.0078,0.000037,NaN,...,0.000014,0.027084,0.000358,0.000018,0.001467,142.39688,295.41414,3.00537,0.155193,3
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000006,0.003589,NaN,NaN,0.000220,NaN,170.80919,1.46903,0.061529,4
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.75586,0.18286,NaN,5


In [16]:
def f2(x):
    if x in range(1, 24):
        return "SKAGERAK"
    elif x in range(24, 91):
        return "NORTH SEA"
    elif x in range(91, 171):
        return "NORWEGIAN SEA2"
    elif x in range(171, 248):
        return "LOFOTEN-BARENTS SEA"
    else:
        return np.nan


# Assign main catchments to OSPAR regions
df["osp_reg"] = df["vass"].apply(f2)

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PCB_tonn,...,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass,osp_reg
0,001,0.019290,1.362000e-02,0.007670,0.085870,0.000000,NaN,NaN,0.043210,NaN,...,0.012184,0.000001,0.000000,0.000801,77.72544,125.01180,1.22541,0.057156,1,SKAGERAK
1,002,0.011934,6.225700e-03,0.899602,2.712862,0.003595,NaN,NaN,0.654341,NaN,...,0.196834,0.000246,0.000000,0.007702,383.96749,729.93910,5.62835,0.737233,2,SKAGERAK
2,003,0.000012,6.000000e-07,0.000054,0.000076,0.000000,0.0,0.0078,0.000037,NaN,...,0.027084,0.000358,0.000018,0.001467,142.39688,295.41414,3.00537,0.155193,3,SKAGERAK
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003589,NaN,NaN,0.000220,NaN,170.80919,1.46903,0.061529,4,SKAGERAK
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.75586,0.18286,NaN,5,SKAGERAK


In [17]:
# Group by OSPAR region
df.fillna(0, inplace=True)
df = df.groupby("osp_reg").sum()
df.drop(0, inplace=True)

# Total for Norway
df.loc["NORWAY"] = df.sum(axis=0)

# Join to model results
df = new_df.join(df)

# Get cols of interest
umod_cols = ["S.P.M.", "TOC", "As", "Pb", "Cd", "Cu", "Zn", "Ni", "Cr", "Hg"]
umod_cols = [
    "%s_%s_tonn" % (i, j) for i in ["INDUSTRI", "RENSEANLEGG"] for j in umod_cols
]
cols = list(new_df.columns) + umod_cols
cols.remove("RENSEANLEGG_TOC_tonn")
df = df[cols]

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,327178,105079,93125,120626,8348
sew_n,13013,1442,3444,2804,5322
sew_p,1100,164,466,383,86
ind_n,2360,125,441,899,895
ind_p,227,11,83,95,38
fish_n,70816,18948,20875,30955,37
fish_p,12210,3260,3594,5349,6
diff_n,31138,4898,11615,11706,2919
diff_p,679,99,195,286,100
sew_po4,660,99,280,230,52


## 7. Fish farm copper

Finally, we need to add in the Cu totals from fish farms. The method is similar to that used above, but simpler because we're only interested in one parameter.

In [18]:
# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]
sql = (
    "SELECT SUBSTR(regine, 1, 3) AS vassdrag, "
    "  SUM(value) AS value FROM ( "
    "    SELECT b.regine, "
    "           c.name, "
    "           (a.value*d.factor) AS value "
    "    FROM resa2.rid_kilder_aqkult_values a, "
    "    resa2.rid_kilder_aquakultur b, "
    "    resa2.rid_punktkilder_outpar_def c, "
    "    resa2.rid_punktkilder_inp_outp d "
    "    WHERE a.anlegg_nr = b.nr "
    "    AND (1, b.regine) NOT IN %s "
    "    AND a.inp_par_id = d.in_pid "
    "    AND c.out_pid = d.out_pid "
    "    AND name = 'Cu' "
    "    AND ar = %s) "
    "GROUP BY SUBSTR(regine, 1, 3)" % (tuple(nd_list_hack), year)
)
aq_df = pd.read_sql(sql, engine)

# Get vassdrag
aq_df["vass"] = aq_df["vassdrag"].apply(f)
aq_df = aq_df.query("vass != -999")

# Calc OSPAR region and group
aq_df["osp_reg"] = aq_df["vass"].apply(f2)
aq_df.fillna(0, inplace=True)
aq_df = aq_df.groupby("osp_reg").sum()
del aq_df["vass"]

# Total for Norway
aq_df.loc["NORWAY"] = aq_df.sum(axis=0)

# Rename
aq_df.columns = [
    "AQUAKULTUR_Cu_tonn",
]

# Join model results
df = df.join(aq_df)

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,327178,105079,93125,120626,8348
sew_n,13013,1442,3444,2804,5322
sew_p,1100,164,466,383,86
ind_n,2360,125,441,899,895
ind_p,227,11,83,95,38
fish_n,70816,18948,20875,30955,37
fish_p,12210,3260,3594,5349,6
diff_n,31138,4898,11615,11706,2919
diff_p,679,99,195,286,100
sew_po4,660,99,280,230,52


In [19]:
# Write output
out_csv = f"../../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
df.to_csv(out_csv)

This data can then be used to create Table 3 in the report - see [this notebook](https://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/summary_table_2017.ipynb) for details.